In [1]:
import os
import gradio as gr

# my OracleVectorStore for LangChain
from oracle_vector_db_lc import OracleVectorStore

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_community.llms import OCIGenAI

from utils import format_docs
from config import EMBED_MODEL, GENAI_MODEL, ENDPOINT

from config_private import COMPARTMENT_OCID

In [2]:
# per il tracing

os.environ["TOKENIZERS_PARALLELISM"] = "false"

TOP_K = 4

embed_model = OCIGenAIEmbeddings(
    auth_type="API_KEY",
    model_id=EMBED_MODEL,
    service_endpoint=ENDPOINT,
    compartment_id=COMPARTMENT_OCID,
)

llm = OCIGenAI(
    auth_type="API_KEY",
    model_id=GENAI_MODEL,
    service_endpoint=ENDPOINT,
    compartment_id=COMPARTMENT_OCID,
    model_kwargs={
        "max_tokens": 1024,
        "temperature": 0.1,
    },
)

# the prompt. This is OK for Cohere
prompt = hub.pull("rlm/rag-prompt")

# build AI Vector Search Vector Store
v_store = OracleVectorStore(
    embedding=embed_model, collection_name="ORACLE_KNOWLEDGE", verbose=True
)

In [3]:
retriever = v_store.as_retriever(search_kwargs={"k": TOP_K})

In [4]:
# using LangChain LCEL language
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [5]:
question = "What is Oracle Strategy for Generative AI?"

In [6]:
%%time
answer = rag_chain.invoke(question)

print(f"Question: {question}")
print("")
print(answer)
print("")

2024-02-27 16:51:39,949 - INFO - top_k: 4
2024-02-27 16:51:39,950 - INFO - 
2024-02-27 16:51:42,436 - INFO - select: select C.id, C.CHUNK, C.REF, 
                            ROUND(VECTOR_DISTANCE(C.VEC, :1, DOT), 3) as d 
                            from ORACLE_KNOWLEDGE C
                            order by d
                            FETCH FIRST 4 ROWS ONLY
2024-02-27 16:51:42,652 - INFO - Query duration: 0.4 sec.


Question: What is Oracle Strategy for Generative AI?

 Oracle's strategy for generative AI centers around providing enterprises with high-performing models at a low cost. They aim to empower businesses with access to best-in-class GPUs and networking, specialized training with their own insights, and seamless integration across their entire cloud of services, from data to applications. Oracle seeks to simplify the use of AI for businesses and improve ROI by offering enterprises an entire stack of integrated services, from out-of-the-box models to AI-infused SaaS applications supported by autonomous databases. 

 oracle also provides its OCI Generative AI service, which can be used on-demand and scaled using the public cloud. Alternatively, they can deliver their services via dedicated regions within client data centers. 

Through these methods, Oracle aims to differentiate its enterprise AI strategy while accommodating businesses regardless of their stage in their AI journey. 

Would y

In [7]:
def answer(question):
    answer = rag_chain.invoke(question)

    return answer


demo = gr.Interface(
    title="Semantic Search on OCI Knowledge",
    fn=answer,
    inputs=[gr.Textbox(lines=2, label="Question")],
    outputs=gr.Textbox(lines=10, interactive=False, label="Answer"),
    allow_flagging="never",
    analytics_enabled=False,
)

demo.launch(share=True)

2024-02-27 16:51:55,305 - INFO - HTTP Request: GET http://127.0.0.1:7860/startup-events "HTTP/1.1 200 OK"
2024-02-27 16:51:55,314 - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7860


2024-02-27 16:51:56,865 - INFO - HTTP Request: GET https://api.gradio.app/v2/tunnel-request "HTTP/1.1 200 OK"
2024-02-27 16:51:57,100 - INFO - HTTP Request: GET https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_darwin_arm64 "HTTP/1.1 200 OK"



Could not create share link. Missing file: /Users/lsaetta/miniforge3/envs/langchain01/lib/python3.9/site-packages/gradio/frpc_darwin_arm64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_darwin_arm64
2. Rename the downloaded file to: frpc_darwin_arm64_v0.2
3. Move the file to this location: /Users/lsaetta/miniforge3/envs/langchain01/lib/python3.9/site-packages/gradio


2024-02-27 16:52:28,791 - INFO - top_k: 4
2024-02-27 16:52:28,792 - INFO - 
2024-02-27 16:52:28,794 - INFO -  2024-02-27 15:52:28.794570: Request: POST https://inference.generativeai.us-chicago-1.oci.oraclecloud.com/20231130/actions/embedText
2024-02-27 16:52:30,634 - INFO - select: select C.id, C.CHUNK, C.REF, 
                            ROUND(VECTOR_DISTANCE(C.VEC, :1, DOT), 3) as d 
                            from ORACLE_KNOWLEDGE C
                            order by d
                            FETCH FIRST 4 ROWS ONLY
2024-02-27 16:52:30,846 - INFO - Query duration: 0.4 sec.
2024-02-27 16:52:30,867 - INFO -  2024-02-27 15:52:30.866994: Request: POST https://inference.generativeai.us-chicago-1.oci.oraclecloud.com/20231130/actions/generateText
2024-02-27 16:53:29,836 - INFO - top_k: 4
2024-02-27 16:53:29,837 - INFO - 
2024-02-27 16:53:29,838 - INFO -  2024-02-27 15:53:29.838933: Request: POST https://inference.generativeai.us-chicago-1.oci.oraclecloud.com/20231130/actions/embedT